<a href="https://colab.research.google.com/github/Atulkhiyani0909/NyayaSetu/blob/main/ChatBot/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [ ]:
!pip install --quiet langchain-core langchain langchain-google-genai pinecone-client langchain-pinecone langchain-huggingface pydantic gradio transformers

## Environment

In [ ]:
from google.colab import userdata
import os

In [ ]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')

os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')

os.environ['PINECONE_API_KEY']=userdata.get('PINECONE_API_KEY')